## Creates the doc2vec vector embeddings for a specific configuration

In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import io
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random
import gzip

from multiprocessing import Process, Queue

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *
from thesis.utils.file import *

## Global variables used throughout the script

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234

In [4]:
MIN_WORD_COUNT = 100
NUM_CORES = 8

In [5]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [6]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"

In [7]:
root_location = "/mnt/virtual-machines/data/"
exports_location = root_location + "exported_data/"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

## Load general data required for classification

In [8]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 18.1 s, sys: 1.11 s, total: 19.3 s
Wall time: 19.3 s


In [9]:
len(training_docs_list)

1286325

In [10]:
len(validation_docs_list)

321473

In [11]:
len(test_docs_list)

401877

# Utility functions for data loading

In [12]:
VALIDATION_MINI_BATCH_SIZE = 10000
def get_extended_docs_with_inference_data_only(doc2vec_model, file_to_write, preprocessed_files_prefix, level, model_name):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_documents_reps = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
    else:
        inference_documents_reps = {}
        info("===== Getting vectors with inference")

        # Multi-threaded inference
#         inference_docs_iterator = ExtendedPVDocumentBatchGenerator(preprocessed_files_prefix, batch_size=None)
        inference_docs_iterator = BatchWrapper(preprocessed_files_prefix, batch_size=None, level=level, level_type=model_name)
        generator_func = inference_docs_iterator.__iter__()
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = VALIDATION_MINI_BATCH_SIZE
        batches_run = 1
        pool = ThreadPool(NUM_CORES)
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {} tags".format(batches_run * mini_batch_size))
            batches_run += 1
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break
                
        pool.close()
        pool.terminate()

        pickle.dump(inference_documents_reps,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_documents_reps

In [13]:
class ExtendedPVDocumentBatchGenerator(Process):
    def __init__(self, filename_prefix, queue, batch_size=10000, start_file=0, offset=10000):
        super(ExtendedPVDocumentBatchGenerator, self).__init__()
        self.queue = queue
        self.offset = offset
        self.filename_prefix = filename_prefix
        self.files_loaded = start_file - offset

    def run(self):
        cur_file = None
        while True:
            try:
                if cur_file is None:
                    info("Loading new file for index: {}".format(str(self.files_loaded + self.offset)))
                    cur_file = io.BufferedReader(gzip.open(self.filename_prefix + str(self.files_loaded + self.offset) + '.gz'))
#                     cur_file = open(self.filename_prefix + str(self.files_loaded + self.offset))
                    self.files_loaded += self.offset
                for line in cur_file:
                    self.queue.put(line)
                cur_file.close()
                cur_file = None
            except IOError:
                self.queue.put(False, block=True, timeout=None)
                info("All files are loaded - last file: {}".format(str(self.files_loaded + self.offset)))
                return


class BatchWrapper(object):
    def __init__(self, training_preprocessed_files_prefix, buffer_size=10000, batch_size=10000, level=1, level_type=None):
        assert batch_size <= 10000 or batch_size is None
        self.level = level
        self.level_type = level_type[0] if level_type is not None else None
        self.batch_size = batch_size
        self.q = Queue(maxsize=buffer_size)
        self.p = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, queue=self.q,
                                                  batch_size=batch_size, start_file=0, offset=10000)
        self.p.start()
        self.cur_data = []

    def is_correct_type(self, doc_id):
        parts = doc_id.split("_")
        len_parts = len(parts)
        if len_parts == self.level:
            if len_parts == 1:
                return True
            if len_parts == self.level and (parts[1][0] == self.level_type or self.level_type is None):
                return True
        return False

    def return_sentences(self, line):
        line_array = tuple(line.split(" "))
        doc_id = line_array[0]
        if not self.is_correct_type(doc_id):
            return False
        line_array = line_array[1:]
        len_line_array = len(line_array)
        # divide the document to batches according to the batch size
        sentences = []
        
        if self.batch_size is None:
            # dont use LabeledSentence for validation iterator
            sentences.append((doc_id, line_array))
        else:
            curr_batch_iter = 0
            while curr_batch_iter < len_line_array:
                sentences.append(LabeledSentence(words=line_array[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id]))
                curr_batch_iter += self.batch_size
        return tuple(sentences)

    def __iter__(self):
        while True:
            item = self.q.get(block=True)
            if item is False:
                self.p.terminate()
                raise StopIteration()
            else:
                sentences = self.return_sentences(item)
                if not sentences:
                    None
                else:
                    for sentence in sentences:
                        yield sentence


# Doc2vec and SVM Parameters

In [14]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 8
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 100000 # report vocab progress every x documents

## Create the Doc2vec model and create/load the vocab

In [15]:
models = [
    (2, 'claims')
]
level, model_name = models[0]

In [16]:
info("creating/loading vocabulary for " + str(level) + ' ' + model_name + ' in ')
doc2vec_model_save_location = os.path.join(root_location,
                                           "parameter_search_doc2vec_models_recalc_" + str(level) + '_' + model_name,
                                           "full")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}_model_{}'.format(DOC2VEC_SIZE,
                                                                DOC2VEC_WINDOW,
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE),
                                                                str(level) + '_' + model_name
                                                                )
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
info("FILE " + os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
doc2vec_model = Doc2Vec(size=DOC2VEC_SIZE, window=DOC2VEC_WINDOW, min_count=MIN_WORD_COUNT,
                max_vocab_size= DOC2VEC_MAX_VOCAB_SIZE,
                sample=DOC2VEC_SAMPLE, seed=DOC2VEC_SEED, workers=NUM_CORES,
                # doc2vec algorithm dm=1 => PV-DM, dm=2 => PV-DBOW, PV-DM dictates CBOW for words
                dm=DOC2VEC_TYPE,
                # hs=0 => negative sampling, hs=1 => hierarchical softmax
                hs=DOC2VEC_HIERARCHICAL_SAMPLE, negative=DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                dm_concat=DOC2VEC_CONCAT,
                # would train words with skip-gram on top of cbow, we don't need that for now
                dbow_words=DOC2VEC_TRAIN_WORDS,
                iter=DOC2VEC_EPOCHS)

GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model


2017-04-19 02:01:21,478 : INFO : creating/loading vocabulary for 2 claims in 
2017-04-19 02:01:21,479 : INFO : FILE /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_claims/full/vocab_model/model


In [17]:
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX)):
    
    info("Creating vocab model")
    training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
    doc2vec_model.build_vocab(sentences=training_docs_iterator, progress_per=REPORT_VOCAB_PROGRESS)
    doc2vec_model.save(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
else:
    info("Loading vocab model")
    doc2vec_model_vocab_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
    doc2vec_model.reset_from(doc2vec_model_vocab_model)

2017-04-19 02:01:25,416 : INFO : Creating vocab model
2017-04-19 02:01:25,455 : INFO : collecting all words and their counts
2017-04-19 02:01:25,457 : INFO : Loading new file for index: 0
2017-04-19 02:01:25,466 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-04-19 02:01:47,435 : INFO : Loading new file for index: 10000
2017-04-19 02:02:08,779 : INFO : Loading new file for index: 20000
2017-04-19 02:02:32,503 : INFO : Loading new file for index: 30000
2017-04-19 02:02:56,733 : INFO : Loading new file for index: 40000
2017-04-19 02:03:20,681 : INFO : Loading new file for index: 50000
2017-04-19 02:03:44,844 : INFO : Loading new file for index: 60000
2017-04-19 02:04:08,454 : INFO : Loading new file for index: 70000
2017-04-19 02:04:32,564 : INFO : Loading new file for index: 80000
2017-04-19 02:04:56,213 : INFO : Loading new file for index: 90000
2017-04-19 02:05:19,723 : INFO : Loading new file for index: 100000
2017-04-19 02:05:20,245 : INFO : PROG

2017-04-19 02:37:32,967 : INFO : PROGRESS: at example #1000000, processed 1082895120 words (546059/s), 1535141 word types, 999495 tags
2017-04-19 02:37:33,362 : INFO : Loading new file for index: 1000000
2017-04-19 02:37:53,204 : INFO : Loading new file for index: 1010000
2017-04-19 02:38:13,986 : INFO : Loading new file for index: 1020000
2017-04-19 02:38:34,521 : INFO : Loading new file for index: 1030000
2017-04-19 02:38:55,123 : INFO : Loading new file for index: 1040000
2017-04-19 02:39:16,304 : INFO : Loading new file for index: 1050000
2017-04-19 02:39:36,095 : INFO : Loading new file for index: 1060000
2017-04-19 02:39:57,258 : INFO : Loading new file for index: 1070000
2017-04-19 02:40:16,749 : INFO : Loading new file for index: 1080000
2017-04-19 02:40:36,479 : INFO : Loading new file for index: 1090000
2017-04-19 02:40:56,435 : INFO : PROGRESS: at example #1100000, processed 1190930945 words (530974/s), 1638926 word types, 1099439 tags
2017-04-19 02:40:56,972 : INFO : Loadin

## Actual Training, validation and Metrics Loop

In [18]:
doc2vec_model.min_alpha = 0.025
DOC2VEC_ALPHA_DECREASE = 0.001

In [19]:
doc2vec_model.workers = NUM_CORES

In [20]:
%%time
# when resuming, resume from an epoch with a previously created doc2vec model to get the learning rate right
start_from = 1
for epoch in range(start_from, DOC2VEC_MAX_EPOCHS+1):
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    info("****************** Epoch {} --- Working on {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        doc2vec_model.workers = NUM_CORES
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        # train the doc2vec model
        training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
        %time doc2vec_model.train(sentences=training_docs_iterator, report_delay=REPORT_DELAY)
        doc2vec_model.alpha -= DOC2VEC_ALPHA_DECREASE  # decrease the learning rate
        doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
        ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME))
        doc2vec_model.save(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
        
    # only do the inference for higher epochs, as inference usually takes as much time as the actual training
    if epoch == 3 or epoch == 5:
        # Validation Embeddings
        info('Getting Validation Embeddings')
        Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix, level, model_name)

2017-04-19 02:47:36,132 : INFO : ****************** Epoch 1 --- Working on doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_claims/epoch_1 *******************
2017-04-19 02:47:36,205 : INFO : training model with 8 workers on 64496 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=10 window=2
2017-04-19 02:47:36,207 : INFO : expecting 1286959 sentences, matching count from corpus used for vocabulary survey
2017-04-19 02:47:36,206 : INFO : Loading new file for index: 0
2017-04-19 02:47:37,245 : INFO : PROGRESS: at 0.02% examples, 198512 words/s, in_qsize 0, out_qsize 0
2017-04-19 02:47:57,267 : INFO : PROGRESS: at 0.53% examples, 226068 words/s, in_qsize 0, out_qsize 0
2017-04-19 02:48:05,374 : INFO : Loading new file for index: 10000
2017-04-19 02:48:17,289 : INFO : PROGRESS: at 1.07% examples, 232515 words/s, in_qsize 0, out_qsize 0
2017-04-19 02:48:34,829 : INFO : Loading new file for index: 20000
2017-04-19 02:48:37,311 : I

2017-04-19 03:05:06,626 : INFO : Loading new file for index: 350000
2017-04-19 03:05:18,153 : INFO : PROGRESS: at 27.46% examples, 231337 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:05:38,180 : INFO : PROGRESS: at 27.94% examples, 230994 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:05:39,316 : INFO : Loading new file for index: 360000
2017-04-19 03:05:58,213 : INFO : PROGRESS: at 28.43% examples, 230749 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:06:11,847 : INFO : Loading new file for index: 370000
2017-04-19 03:06:18,235 : INFO : PROGRESS: at 28.90% examples, 230369 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:06:38,241 : INFO : PROGRESS: at 29.41% examples, 230431 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:06:42,959 : INFO : Loading new file for index: 380000
2017-04-19 03:06:58,242 : INFO : PROGRESS: at 29.89% examples, 230195 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:07:14,509 : INFO : Loading new file for index: 390000
2017-04-19 03:07:18,252 : INFO : PROGRESS:

2017-04-19 03:25:01,180 : INFO : Loading new file for index: 650000
2017-04-19 03:25:19,399 : INFO : PROGRESS: at 50.80% examples, 200944 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:25:39,412 : INFO : PROGRESS: at 51.13% examples, 200457 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:25:48,899 : INFO : Loading new file for index: 660000
2017-04-19 03:25:59,425 : INFO : PROGRESS: at 51.46% examples, 199997 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:26:19,498 : INFO : PROGRESS: at 51.80% examples, 199573 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:26:36,021 : INFO : Loading new file for index: 670000
2017-04-19 03:26:39,518 : INFO : PROGRESS: at 52.12% examples, 199133 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:26:59,518 : INFO : PROGRESS: at 52.45% examples, 198692 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:27:19,552 : INFO : PROGRESS: at 52.78% examples, 198245 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:27:23,639 : INFO : Loading new file for index: 680000
2017-04

2017-04-19 03:46:01,217 : INFO : PROGRESS: at 71.27% examples, 181893 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:46:14,450 : INFO : Loading new file for index: 920000
2017-04-19 03:46:21,248 : INFO : PROGRESS: at 71.61% examples, 181698 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:46:41,257 : INFO : PROGRESS: at 71.95% examples, 181508 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:47:00,780 : INFO : Loading new file for index: 930000
2017-04-19 03:47:01,293 : INFO : PROGRESS: at 72.28% examples, 181334 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:47:21,323 : INFO : PROGRESS: at 72.62% examples, 181147 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:47:41,336 : INFO : PROGRESS: at 72.96% examples, 180975 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:47:46,765 : INFO : Loading new file for index: 940000
2017-04-19 03:48:01,347 : INFO : PROGRESS: at 73.30% examples, 180818 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:48:21,354 : INFO : PROGRESS: at 73.63% examples, 180663 words

2017-04-19 04:06:42,506 : INFO : PROGRESS: at 92.03% examples, 173344 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:07:02,564 : INFO : PROGRESS: at 92.37% examples, 173237 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:07:10,120 : INFO : Loading new file for index: 1190000
2017-04-19 04:07:22,595 : INFO : PROGRESS: at 92.70% examples, 173138 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:07:42,617 : INFO : PROGRESS: at 93.04% examples, 173053 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:07:55,789 : INFO : Loading new file for index: 1200000
2017-04-19 04:08:02,636 : INFO : PROGRESS: at 93.39% examples, 172979 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:08:22,655 : INFO : PROGRESS: at 93.71% examples, 172861 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:08:42,328 : INFO : Loading new file for index: 1210000
2017-04-19 04:08:42,689 : INFO : PROGRESS: at 94.05% examples, 172758 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:09:02,703 : INFO : PROGRESS: at 94.38% examples, 172661 wo

...

2017-04-19 14:54:36,027 : INFO : PROGRESS: at 71.02% examples, 191133 words/s, in_qsize 0, out_qsize 0
2017-04-19 14:54:56,038 : INFO : PROGRESS: at 71.46% examples, 191142 words/s, in_qsize 0, out_qsize 0
2017-04-19 14:54:57,777 : INFO : Loading new file for index: 920000
2017-04-19 14:55:16,060 : INFO : PROGRESS: at 71.90% examples, 191143 words/s, in_qsize 0, out_qsize 0
2017-04-19 14:55:33,581 : INFO : Loading new file for index: 930000
2017-04-19 14:55:36,066 : INFO : PROGRESS: at 72.33% examples, 191176 words/s, in_qsize 0, out_qsize 0
2017-04-19 14:55:56,067 : INFO : PROGRESS: at 72.77% examples, 191181 words/s, in_qsize 0, out_qsize 0
2017-04-19 14:56:09,538 : INFO : Loading new file for index: 940000
2017-04-19 14:56:16,079 : INFO : PROGRESS: at 73.19% examples, 191173 words/s, in_qsize 0, out_qsize 0
2017-04-19 14:56:36,101 : INFO : PROGRESS: at 73.62% examples, 191191 words/s, in_qsize 0, out_qsize 0
2017-04-19 14:56:45,421 : INFO : Loading new file for index: 950000
2017-04

2017-04-19 15:14:17,202 : INFO : PROGRESS: at 96.31% examples, 191149 words/s, in_qsize 0, out_qsize 0
2017-04-19 15:14:20,249 : INFO : Loading new file for index: 1240000
2017-04-19 15:14:37,213 : INFO : PROGRESS: at 96.75% examples, 191167 words/s, in_qsize 0, out_qsize 0
2017-04-19 15:14:56,152 : INFO : Loading new file for index: 1250000
2017-04-19 15:14:57,233 : INFO : PROGRESS: at 97.17% examples, 191163 words/s, in_qsize 0, out_qsize 0
2017-04-19 15:15:17,264 : INFO : PROGRESS: at 97.61% examples, 191171 words/s, in_qsize 0, out_qsize 0
2017-04-19 15:15:32,082 : INFO : Loading new file for index: 1260000
2017-04-19 15:15:37,278 : INFO : PROGRESS: at 98.04% examples, 191164 words/s, in_qsize 0, out_qsize 0
2017-04-19 15:15:57,288 : INFO : PROGRESS: at 98.49% examples, 191218 words/s, in_qsize 0, out_qsize 0
2017-04-19 15:16:07,674 : INFO : Loading new file for index: 1270000
2017-04-19 15:16:17,297 : INFO : PROGRESS: at 98.92% examples, 191218 words/s, in_qsize 0, out_qsize 0
201

CPU times: user 2h 49min 45s, sys: 4min 5s, total: 2h 53min 50s
Wall time: 1h 17min 55s


2017-04-19 15:17:08,530 : INFO : not storing attribute syn0norm
2017-04-19 15:17:08,531 : INFO : storing np array 'syn0' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_claims/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_claims/epoch_8/model.wv.syn0.npy
2017-04-19 15:17:08,568 : INFO : storing np array 'syn1neg' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_claims/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_claims/epoch_8/model.syn1neg.npy
2017-04-19 15:17:08,602 : INFO : not storing attribute cum_table
2017-04-19 15:17:20,674 : INFO : saved /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_claims/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_claims/epoch_8/model


CPU times: user 1d 4h 31min 30s, sys: 49min 33s, total: 1d 5h 21min 3s
Wall time: 12h 29min 44s


## Inference Only (if needed)

In [37]:
NUM_CORES = 32

In [38]:
epoch = 1
GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)

if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
    doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
    doc2vec_model.workers = NUM_CORES
    GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

2017-04-18 18:06:29,959 : INFO : loading Doc2Vec object from /mnt/virtual-machines/data/parameter_search_doc2vec_models_2_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_1/model
2017-04-18 18:06:38,844 : INFO : loading docvecs recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_2_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_1/model.docvecs.* with mmap=None
2017-04-18 18:06:38,845 : INFO : loading doctag_syn0 from /mnt/virtual-machines/data/parameter_search_doc2vec_models_2_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_1/model.docvecs.doctag_syn0.npy with mmap=None
2017-04-18 18:06:39,463 : INFO : loading wv recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_2_description/full/doc2vec_size_200_w_2_

In [39]:
GLOBAL_VARS.MODEL_NAME

'doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_1'

In [ ]:
Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix, level, model_name)

2017-04-18 18:06:48,123 : INFO : ===== Getting vectors with inference
2017-04-18 18:06:48,350 : INFO : Loading new file for index: 0
2017-04-18 18:07:10,806 : INFO : Loading new file for index: 10000
2017-04-18 18:10:51,895 : INFO : Finished: 10000 tags
2017-04-18 18:11:11,459 : INFO : Loading new file for index: 20000
2017-04-18 18:14:42,427 : INFO : Finished: 20000 tags
2017-04-18 18:15:02,688 : INFO : Loading new file for index: 30000
2017-04-18 18:18:25,861 : INFO : Finished: 30000 tags
2017-04-18 18:18:45,916 : INFO : Loading new file for index: 40000
2017-04-18 18:22:10,180 : INFO : Finished: 40000 tags
2017-04-18 18:22:30,523 : INFO : Loading new file for index: 50000
2017-04-18 18:25:47,360 : INFO : Finished: 50000 tags
2017-04-18 18:26:07,038 : INFO : Loading new file for index: 60000
2017-04-18 18:29:19,960 : INFO : Finished: 60000 tags
2017-04-18 18:29:40,442 : INFO : Loading new file for index: 70000
2017-04-18 18:32:57,714 : INFO : Finished: 70000 tags
2017-04-18 18:33:16,

#### Testing inference

In [55]:
inference_docs_iterator = BatchWrapper(validation_preprocessed_files_prefix, batch_size=None, level=level, level_type=model_name)        
for doc_tuple in inference_doczs_iterator:
    doc_id, doc_tokens = doc_tuple
    rep = doc2vec_model.infer_vector(doc_tokens)
    print (doc_id, rep)
    break

('08521002', array([  1.13558674e+00,  -2.01971769e-01,  -9.30447519e-01,
         9.55632687e-01,   5.11517346e-01,   4.34441900e+00,
        -3.77764761e-01,  -1.11617422e+00,  -2.15896085e-01,
         9.09354746e-01,   5.74674904e-01,  -2.07049704e+00,
        -7.20400810e-01,   4.94136661e-01,  -1.74060893e+00,
        -2.17272949e+00,  -4.39270258e-01,  -1.51936769e+00,
         5.65607429e-01,  -4.58835810e-01,  -1.69598356e-01,
         1.77733886e+00,   3.66123140e-01,   1.38953611e-01,
        -1.04259264e+00,   8.84979665e-01,  -8.56729895e-02,
        -6.04329109e-01,   4.42179322e-01,   1.08561194e+00,
        -2.49654725e-01,   3.02951038e-01,  -3.80307257e-01,
         1.32433748e+00,   7.18038738e-01,   7.99864233e-01,
        -3.60305488e-01,  -3.32749695e-01,   1.86409019e-02,
        -1.16298962e+00,  -2.36521304e-01,   8.52507114e-01,
        -4.25269688e-03,  -2.73190904e+00,  -1.39228487e+00,
         3.14658254e-01,   1.19927609e+00,  -8.86219382e-01,
        -2.

In [53]:
doc2vec_model.wv.syn0

array([[ 0.24962339,  0.03808838, -0.38492572, ...,  0.81019139,
        -0.0872335 ,  0.00503489],
       [-0.24393913, -0.9072656 , -0.08245134, ..., -0.12438237,
        -0.10501056,  0.07241193],
       [ 0.06769085, -0.22004843,  0.05649997, ...,  0.15331532,
        -0.87121236, -0.71148068],
       ..., 
       [ 0.02257917,  0.18380728, -0.19475998, ...,  0.72972393,
        -0.03356596, -0.29145467],
       [-0.20255305, -0.25994578,  0.31640032, ..., -0.02623975,
         0.41660461, -0.45980361],
       [ 1.09419656, -0.97489876, -0.3509953 , ...,  0.82430571,
         0.02756385,  0.7905944 ]], dtype=float32)